In [1]:
import polars as pl

df = pl.read_csv("fever_test.filtered.sampled.csv")
df.head()

id,cid,premise,hypothesis,alternative hypothesis,label,new hypothesis,new label,change type
i64,i64,str,str,str,str,str,str,str
0,58846,"""Johnny Galecki . He is known f…","""Johnny Galecki has been in at …","""""","""NEUTRAL""","""""","""""",""""""
1,172460,"""Matteo Renzi ( [ matˈtɛːo ˈrɛn…","""Matteo Renzi served as Prime M…","""""","""ENTAILMENT""","""""","""""",""""""
2,181201,"""Southpaw is a 2015 American sp…","""Southpaw was released 2011.""","""""","""CONTRADICTION""","""""","""""",""""""
3,174024,"""Pink Floyd . The final Pink Fl…","""The Endless River is Pink Floy…","""""","""CONTRADICTION""","""""","""""",""""""
4,118068,"""Beatles Day . This day is cons…","""Liverpool is unrelated to The …","""""","""CONTRADICTION""","""""","""""",""""""


In [31]:
to_skip = df.select('hypothesis').to_numpy().squeeze()
print(to_skip)

['Johnny Galecki has been in at least two French sitcoms.'
 'Matteo Renzi served as Prime Minister of Italy.'
 'Southpaw was released 2011.'
 "The Endless River is Pink Floyd's first studio album."
 'Liverpool is unrelated to The Beatles.'
 "Elizabeth of York wouldn't marry the king."
 'Roberto Orci worked with others.'
 'Yemen is not the second-largest country in the Arabian peninsula.'
 'Military deception only overlaps with psychological peace.'
 'Kenneth Lonergan is a Catholic.'
 "DJ Quik's dog's name is David Marvin Bark." 'Moesia included a monarch.'
 'Michigan is the least populous of the 50 United States.'
 'Jiang Wen is a film series.'
 'Edmund H. North was born in New York on March 12, 1911.'
 'Magic Johnson played basketball.' 'GLOW has one episode.'
 'Younger premiered in 2015.' 'Trollhunters was created by a horse.'
 'Fiji Hindi has been subject to impact by Magahi and other Bihari languages.'
 'Foot Locker is a exclusively Vietnamese retailer.'
 'A Golden Globe was won by

In [17]:
from datasets import load_dataset

dataset = load_dataset("tommasobonomo/sem_augmented_fever_nli", download_mode="force_redownload")
print(dataset['test'])

Generating train split:   0%|          | 0/51086 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2288 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2287 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
    num_rows: 2287
})


In [35]:
to_keep = {
    'cid':[], 
    'premise':[], 
    'hypothesis': [], 
    'label': []
}
from tqdm import tqdm
for split in ['test', 'validation']:
    for sample in tqdm(dataset[split], desc=f"> processing {split} data"):
        #print(sample['id'])
        if sample['premise'] == '' or sample['hypothesis'] == '': #or sample['id'] in to_skip:
            continue
        if sample['hypothesis'] in to_skip:
            continue
        to_keep['cid'].append(sample['id'])
        to_keep['premise'].append(sample['premise'])
        to_keep['hypothesis'].append(sample['hypothesis'])
        to_keep['label'].append(sample['label'])

to_keep_df = pl.from_dict(to_keep)
to_keep_df.height

> processing validation data: 100%|██████████| 2288/2288 [00:01<00:00, 1455.14it/s]


4423

In [36]:
sampled_e = to_keep_df.filter(pl.col('label') == 'ENTAILMENT').sample(n=70, seed=42)
sampled_n = to_keep_df.filter(pl.col('label') == 'NEUTRAL').sample(n=70, seed=42)
sampled_c = to_keep_df.filter(pl.col('label') == 'CONTRADICTION').sample(n=70, seed=42)
print(sampled_e.head())
print(sampled_n.head())
print(sampled_c.head())
concat = pl.concat([sampled_e, sampled_n, sampled_c]).select(pl.all().shuffle(seed=42))
print(concat)

shape: (5, 4)
┌────────┬─────────────────────────────────┬─────────────────────────────────┬────────────┐
│ cid    ┆ premise                         ┆ hypothesis                      ┆ label      │
│ ---    ┆ ---                             ┆ ---                             ┆ ---        │
│ str    ┆ str                             ┆ str                             ┆ str        │
╞════════╪═════════════════════════════════╪═════════════════════════════════╪════════════╡
│ 36333  ┆ Underdog (film) . Directed by … ┆ Underdog was directed by a Bel… ┆ ENTAILMENT │
│ 219708 ┆ Corsica . The island formed a … ┆ Corsica contains a city.        ┆ ENTAILMENT │
│ 61709  ┆ Hollow Man . A direct-to-video… ┆ Hollow Man has a direct-to-vid… ┆ ENTAILMENT │
│ 122252 ┆ Mirny was a 20-gun sloop-of-wa… ┆ The Mirny (sloop-of-war) was i… ┆ ENTAILMENT │
│ 157183 ┆ Juventus F.C. . The club is th… ┆ Juventus F.C. has long worn a … ┆ ENTAILMENT │
└────────┴─────────────────────────────────┴──────────────────────

In [37]:
concat.write_csv("200_FEVER_dev_samples.csv", separator=',')
print("csv written.")

csv written.
